In [ ]:
import cobra.io
import escher
from escher import Builder
from cobra import Model, Reaction, Metabolite
from os.path import join

In [2]:
data_dir = "/Users/phivri/Documents/GitHub/Biosustain/MoGeoModel"

In [3]:
Cljung_Model = cobra.io.read_sbml_model(join(data_dir,'Models/C-ljungdahlii_iHN637.xml'))

medium = Cljung_Model.medium
medium["EX_fru_e"]= 0
medium["EX_co2_e"]= 1000
medium["EX_co_e"]= 0
medium["EX_h2_e"]= 10
Cljung_Model.medium = medium
#solution = Cljung_Model.optimize()

Using license file /Users/phivri/gurobi.lic
Academic license - for non-commercial use only - expires 2022-07-23


# Determine minimal set of reactions for acetogenesis:

In [90]:
Cljung_Model = cobra.io.read_sbml_model(join(data_dir,'Models/C-ljungdahlii_iHN637.xml'))

medium = Cljung_Model.medium
medium["EX_fru_e"]= 0
medium["EX_co2_e"]= 1000
medium["EX_co_e"]= 10
medium["EX_h2_e"]= 0
Cljung_Model.medium = medium

Cljung_Model.reactions.ATPM.bounds = 0, 1000

Cljung_Model.reactions.ACACT1r.bounds = 0,1000.0
Cljung_Model.reactions.HACD1.bounds = 0,1000.0
Cljung_Model.reactions.ECOAH1.bounds = 0,1000.0
Cljung_Model.reactions.ACOAD1z.bounds = 0,1000.0
Cljung_Model.reactions.PBUTT.bounds = 0,1000.0
Cljung_Model.reactions.BUTKr.bounds = -1000.0,0
Cljung_Model.reactions.BTCOARx.bounds = 0,1000.0
Cljung_Model.reactions.ALCD4.bounds = -1000.0,0
Cljung_Model.reactions.BTOHt.bounds = -1000.0, 1000.0
Cljung_Model.reactions.BUTt.bounds = -1000.0, 1000.0

Cljung_Model.reactions.ACt2r.knock_out()

acT = Reaction("ACt")
acT.name = "Acetate transporter"
acT.add_metabolites({ 
    Cljung_Model.metabolites.ac_c: -1.0,
    Cljung_Model.metabolites.ac_e: 1.0
})
acT

#Cljung_Model.reactions.ACACT1r.knock_out()

Cljung_Model.add_reaction(acT)

#Cljung_Model.objective = "ATPS4r"
Cljung_Model.objective = "EX_ac_e"
#Cljung_Model.objective = "EX_etoh_e"
#Cljung_Model.objective = "EX_but_e"
#Cljung_Model.objective = "EX_btoh_e"
#Cljung_Model.objective = "EX_btd_RR_e"
print(Cljung_Model.slim_optimize())
solution = Cljung_Model.optimize()

solution.fluxes[solution.fluxes != 0].index

2.5


Index(['EX_h2o_e', 'EX_ac_e', 'EX_co_e', 'EX_co2_e', 'ACKr', 'ATPM', 'EX_h_e',
       'CO2t', 'COt', 'ALCD4y', 'CODH4', 'CODH_ACS', 'H2Ot', 'FDH7', 'FTHFLi',
       'METR', 'MTHFD', 'MTHFC', 'MTHFR5', 'PTAr', 'RNF', 'ALCD4', 'ATPS4r',
       'ACt'],
      dtype='object')

In [91]:
Builder(model = Cljung_Model, map_json = join(data_dir,'Models/EscherMaps/WL_core_clean.json'), reaction_data = solution.fluxes.to_dict())

Builder(reaction_data={'EX_h2_e': 0.0, 'EX_h2o_e': -5.0, 'EX_his__L_e': 0.0, 'EX_ile__L_e': 0.0, 'EX_lac__D_e'…

In [80]:
WLmodel = Model('Wood_Ljungdahl_Core')

WLmodel.add_reactions([
Cljung_Model.reactions.EX_h2_e,
Cljung_Model.reactions.EX_h2o_e,
Cljung_Model.reactions.EX_ac_e,
Cljung_Model.reactions.EX_co2_e,
Cljung_Model.reactions.EX_co_e,
Cljung_Model.reactions.EX_h_e,
Cljung_Model.reactions.CO2t,
Cljung_Model.reactions.COt,
Cljung_Model.reactions.ACt,

Cljung_Model.reactions.ACKr,
Cljung_Model.reactions.ADK1,
Cljung_Model.reactions.H2Ot,
Cljung_Model.reactions.H2td,

Cljung_Model.reactions.ASPT,
Cljung_Model.reactions.CODH4,
Cljung_Model.reactions.CODH_ACS,

Cljung_Model.reactions.FRNDPR2r,
Cljung_Model.reactions.POR,
Cljung_Model.reactions.PFL,
    
Cljung_Model.reactions.FDH7,
Cljung_Model.reactions.FUM,
Cljung_Model.reactions.FTHFLi,
Cljung_Model.reactions.GLUDy,
Cljung_Model.reactions.METR,
Cljung_Model.reactions.HYDFDN2r,
Cljung_Model.reactions.MDH,
Cljung_Model.reactions.MTHFD,
Cljung_Model.reactions.MTHFC,
Cljung_Model.reactions.MTHFR5,
Cljung_Model.reactions.PTAr,
Cljung_Model.reactions.RNF,
Cljung_Model.reactions.PPA,
Cljung_Model.reactions.ASPTA,
Cljung_Model.reactions.ATPS4r,
Cljung_Model.reactions.NTPP6])

WLmodel.objective = "EX_ac_e"
print(WLmodel.slim_optimize())
solution = WLmodel.optimize()
Builder(model = WLmodel, map_json = join(data_dir,'Models/EscherMaps/WL_core.json'), reaction_data = solution.fluxes.to_dict())

2.5


Builder(reaction_data={'EX_h2_e': 0.0, 'EX_h2o_e': -5.0, 'EX_ac_e': 2.5, 'EX_co2_e': 5.0, 'EX_co_e': -10.0, 'E…

In [57]:
for x in Cljung_Model.reactions:
    if x.bounds == (0, 0):{
        print(x.id)
    }

ATPM
BTOHt
BUTt
HYDFDi
HYDFDN


In [92]:
with Cljung_Model:
    Cljung_Model.objective = "EX_ac_e"
    Cljung_Model.reactions.ATPM.bounds = 0, 0
    print("Acetate production")
    print("Product flux", Cljung_Model.slim_optimize())
    print("ATP flux", Cljung_Model.reactions.ATPS4r.flux)
    print("CO2 flux", Cljung_Model.reactions.CO2t.flux)
    
with Cljung_Model:
    Cljung_Model.objective = "EX_etoh_e"
    Cljung_Model.reactions.ATPM.bounds = 0, 0
    print("Ethanol production")
    print("Product flux", Cljung_Model.slim_optimize())
    print("ATP flux", Cljung_Model.reactions.ATPS4r.flux)
    print("CO2 flux", Cljung_Model.reactions.CO2t.flux)

with Cljung_Model:
    Cljung_Model.objective = "EX_but_e"
    Cljung_Model.reactions.ATPM.bounds = 0, 0
    Cljung_Model.reactions.BUTt.bounds = -1000.0, 1000.0
    Cljung_Model.reactions.BUTKr.bounds = -1000.0, 1000.0
    Cljung_Model.reactions.PBUTT.bounds = -1000.0, 1000.0
    Cljung_Model.reactions.ACOAD1z.bounds = -1000.0, 1000.0
    Cljung_Model.reactions.ECOAH1.bounds = -1000.0, 1000.0
    Cljung_Model.reactions.HACD1.bounds = -1000.0, 1000.0
    Cljung_Model.reactions.ACACT1r.bounds = -1000.0, 1000.0
    print("Butyrate production")
    print("Product flux", Cljung_Model.slim_optimize())
    print("ATP flux", Cljung_Model.reactions.ATPS4r.flux)
    print("CO2 flux", Cljung_Model.reactions.CO2t.flux)
    
with Cljung_Model:
    Cljung_Model.objective = "EX_btoh_e"
    Cljung_Model.reactions.ATPM.bounds = 0, 0
    Cljung_Model.reactions.BTOHt.bounds = -1000.0, 1000.0
    Cljung_Model.reactions.ALCD4.bounds = -1000.0, 1000.0
    Cljung_Model.reactions.BTCOARx.bounds = -1000.0, 1000.0
    Cljung_Model.reactions.ACOAD1z.bounds = -1000.0, 1000.0
    Cljung_Model.reactions.ECOAH1.bounds = -1000.0, 1000.0
    Cljung_Model.reactions.HACD1.bounds = -1000.0, 1000.0
    Cljung_Model.reactions.ACACT1r.bounds = -1000.0, 1000.0
    print("Butanol production")
    print("Product flux", Cljung_Model.slim_optimize())
    print("ATP flux", Cljung_Model.reactions.ATPS4r.flux)
    print("CO2 flux", Cljung_Model.reactions.CO2t.flux)
    
with Cljung_Model:
    Cljung_Model.objective = "EX_btd_RR_e"
    Cljung_Model.reactions.ATPM.bounds = 0, 0
    Cljung_Model.reactions.BTDD_RR.bounds = -1000.0, 1000.0
    Cljung_Model.reactions.BTDt_RR.bounds = -1000.0, 1000.0
    Cljung_Model.reactions.ACLDC.bounds = -1000.0, 1000.0
    Cljung_Model.reactions.ACLS.bounds = -1000.0, 1000.0
    Cljung_Model.reactions.POR.bounds = -1000.0, 1000.0
    print("Butanediol production")
    print("Product flux", Cljung_Model.slim_optimize())
    print("ATP flux", Cljung_Model.reactions.ATPS4r.flux)
    print("CO2 flux", Cljung_Model.reactions.CO2t.flux)

Acetate production
Product flux 2.5
ATP flux 2.5
CO2 flux -5.0
Ethanol production
Product flux 1.666666666666666
ATP flux 2.9166666666666665
CO2 flux -6.666666666666668
Butyrate production
Product flux 1.0
ATP flux 3.75
CO2 flux -6.0
Butanol production
Product flux 0.8333333333333335
ATP flux 4.166666666666669
CO2 flux -6.666666666666668
Butanediol production
Product flux 0.9090909090909091
ATP flux 2.7272727272727266
CO2 flux -6.3636363636363615


In [ ]:
Builder(model = Cljung_Model, map_json = join(data_dir,'Models/EscherMaps/WL_core.json'), reaction_data = solution.fluxes.to_dict())

In [ ]:
Cljung_Model.reactions.EX_h2_e
Cljung_Model.reactions.EX_h2o_e
Cljung_Model.reactions.EX_ac_e
Cljung_Model.reactions.EX_co2_e
Cljung_Model.reactions.ACt2r
Cljung_Model.reactions.ACKr
Cljung_Model.reactions.ADK1
Cljung_Model.reactions.ATPM
Cljung_Model.reactions.EX_h_e
Cljung_Model.reactions.CO2t
Cljung_Model.reactions.CODH_ACS
Cljung_Model.reactions.H2Ot
Cljung_Model.reactions.H2td
Cljung_Model.reactions.FDH7
Cljung_Model.reactions.FRNDPR2r
Cljung_Model.reactions.FTHFLi
Cljung_Model.reactions.METR
Cljung_Model.reactions.HYDFDN2r
Cljung_Model.reactions.MTHFD
Cljung_Model.reactions.MTHFC
Cljung_Model.reactions.MTHFR5
Cljung_Model.reactions.PTAr
Cljung_Model.reactions.RNF
Cljung_Model.reactions.PPA
Cljung_Model.reactions.ATPS4r
Cljung_Model.reactions.NTPP6

In [9]:
WLmodel = Model('Wood_Ljungdahl_Core')

WLmodel.add_reactions([
#Transport & Exchange:
Cljung_Model.reactions.EX_h2_e,
Cljung_Model.reactions.EX_h2o_e,
Cljung_Model.reactions.EX_co2_e,
Cljung_Model.reactions.EX_co_e,
Cljung_Model.reactions.EX_ac_e,
Cljung_Model.reactions.EX_etoh_e,
Cljung_Model.reactions.EX_btd_RR_e,
Cljung_Model.reactions.EX_but_e,
Cljung_Model.reactions.EX_btoh_e,
Cljung_Model.reactions.H2td,
Cljung_Model.reactions.H2Ot,
Cljung_Model.reactions.CO2t,
Cljung_Model.reactions.COt,
#Cljung_Model.reactions.ACt2r,
Cljung_Model.reactions.ETOHt,
Cljung_Model.reactions.BTDt_RR,
Cljung_Model.reactions.BUTt,
Cljung_Model.reactions.BTOHt,
#Energy:
Cljung_Model.reactions.HYDFDN,
Cljung_Model.reactions.HYDFDi,
Cljung_Model.reactions.HYDFDN2r,
Cljung_Model.reactions.ATPS4r,
Cljung_Model.reactions.ATPM,
Cljung_Model.reactions.RNF,#Add this reaction in Escher
Cljung_Model.reactions.NTPP6,#Add this reaction in Escher
Cljung_Model.reactions.PPA,#Add this reaction in Escher
Cljung_Model.reactions.ADK1,#Add this reaction in Escher
Cljung_Model.reactions.FRNDPR2r,#Add this reaction in Escher
#WL pathway
Cljung_Model.reactions.FDH7,
Cljung_Model.reactions.FTHFLi,
Cljung_Model.reactions.MTHFC,
Cljung_Model.reactions.MTHFD,
Cljung_Model.reactions.MTHFR5,
Cljung_Model.reactions.METR,
Cljung_Model.reactions.CODH4,
Cljung_Model.reactions.CODH_ACS,
Cljung_Model.reactions.PTAr,
Cljung_Model.reactions.ACKr,
Cljung_Model.reactions.ACAFDOR,
Cljung_Model.reactions.ALCD2y,
Cljung_Model.reactions.ALCD2x,
Cljung_Model.reactions.POR,
Cljung_Model.reactions.ACLS,
Cljung_Model.reactions.ACLDC,
Cljung_Model.reactions.BTDD_RR,
Cljung_Model.reactions.ACACT1r,
Cljung_Model.reactions.HACD1,
Cljung_Model.reactions.ECOAH1,
Cljung_Model.reactions.ACOAD1z,
Cljung_Model.reactions.BTCOARx,
Cljung_Model.reactions.ALCD4,
Cljung_Model.reactions.PBUTT,
Cljung_Model.reactions.BUTKr])

In [10]:
for x in WLmodel.reactions:
    if x.bounds == (0, 0):{
        print(x.id)
    }

BUTt
BTOHt
HYDFDN
HYDFDi
ATPM
ACACT1r
HACD1
ECOAH1
ACOAD1z
BTCOARx
ALCD4
PBUTT
BUTKr


In [11]:
WLmodel.objective = "EX_etoh_e"
WLmodel.optimize()

,fluxes,reduced_costs
EX_h2_e,-10.000000,-0.333333
EX_h2o_e,5.000000,0.000000
EX_co2_e,-3.333333,-0.000000
EX_co_e,0.000000,-0.333333
EX_ac_e,0.000000,-2.333333
...,...,...
ACOAD1z,0.000000,-0.333333
BTCOARx,0.000000,-1.666667
ALCD4,0.000000,0.333333
PBUTT,0.000000,-2.333333


In [13]:
WLmodel.reactions.ACt2r

Reaction identifier,ACt2r
Name,Acetate reversible transport via proton symport
Memory address,0x07fe190decfd0
Stoichiometry,ac_e + h_e <=> ac_c + h_c Acetate + H+ <=> Acetate + H+
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [12]:
WLmodel.objective = "EX_ac_e"
WLmodel.optimize()
#solution = WLmodel.optimize()
WLmodel.summary()
#print("Flux through objective function: ",WLmodel.objective.expression, " = ",round(WLmodel.slim_optimize(),1))
#Builder(model = WLmodel, map_json = join(data_dir,'Models/EscherMaps/WL_core.json'), reaction_data = solution.fluxes.to_dict())

Metabolite,Reaction,Flux,C-Number,C-Flux
Metabolite,Reaction,Flux,C-Number,C-Flux


# Add Acetone production pathway

In [14]:
WLmodelActn = WLmodel

WLmodelActn.add_metabolites(Metabolite(id='acac_c',formula = 'C4H6O3', name = 'Acetoacetate', compartment = 'c' ))
WLmodelActn.add_metabolites(Metabolite(id='act_c',formula = 'C3H6O', name = 'Acetone', compartment = 'c' ))
WLmodelActn.add_metabolites(Metabolite(id='act_e', formula = 'C3H6O', name = 'Acetone', compartment = 'e'))

In [15]:
ctfAB_ac = Reaction("ctfAB_ac")
ctfAB_ac.name = "Acetate-acetoacetate CoA-transferase"

ctfAB_ac.add_metabolites({
    WLmodelActn.metabolites.ac_c: -1.0,      #Acetate
    WLmodelActn.metabolites.aacoa_c: -1.0,   #Acetoacetyl-CoA
    WLmodelActn.metabolites.acac_c: 1.0,     #Acetoacetate
    WLmodelActn.metabolites.accoa_c: 1.0,    #Acetyl-CoA
    WLmodelActn.metabolites.h_c: 1.0
})
print(ctfAB_ac.check_mass_balance())    #ensure correct mass balance
print(ctfAB_ac.bounds)                  #Ensure correct bounds
print(ctfAB_ac.reaction)                #Ensure correct reaction

{'H': 2.0}
(0.0, 1000.0)
aacoa_c + ac_c --> acac_c + accoa_c + h_c


In [16]:
adc = Reaction("ADC")
adc.name = "acetoacetate decarboxylase"

adc.add_metabolites({
    WLmodelActn.metabolites.acac_c: -1.0,     #Acetoacetate
    WLmodelActn.metabolites.h_c: -1.0, 
    WLmodelActn.metabolites.act_c: 1.0,
    WLmodelActn.metabolites.co2_c: 1.0
})
print(adc.check_mass_balance())    #ensure correct mass balance
print(adc.reaction)                #Ensure correct reaction

{'H': -1.0}
acac_c + h_c --> act_c + co2_c


In [17]:
acT = Reaction("ACt")
acT.name = "Acetate transporter"
acT.add_metabolites({ 
    WLmodelActn.metabolites.ac_c: -1.0,
    WLmodelActn.metabolites.ac_e: 1.0
})
acT

Reaction identifier,ACt
Name,Acetate transporter
Memory address,0x07fe1904bba20
Stoichiometry,ac_c --> ac_e Acetate --> Acetate
GPR,
Lower bound,0.0
Upper bound,1000.0


In [18]:
actT = Reaction("ACTt")
actT.name = "Acetone transporter"
actT.add_metabolites({ 
    WLmodelActn.metabolites.act_c: -1.0,
    WLmodelActn.metabolites.act_e: 1.0
})
actT

Reaction identifier,ACTt
Name,Acetone transporter
Memory address,0x07fe190496e48
Stoichiometry,act_c --> act_e Acetone --> Acetone
GPR,
Lower bound,0.0
Upper bound,1000.0


In [19]:
act_ex = Reaction("EX_act_e")
act_ex.name = "Acetone exchange"
act_ex.add_metabolites({ 
    WLmodelActn.metabolites.act_e: -1.0,
})
act_ex

Reaction identifier,EX_act_e
Name,Acetone exchange
Memory address,0x07fe190492a90
Stoichiometry,act_e --> Acetone -->
GPR,
Lower bound,0.0
Upper bound,1000.0


In [20]:
WLmodelActn.add_reactions([ctfAB_ac, adc, acT, actT, act_ex])

In [21]:
WLmodelActn.objective = "EX_etoh_e"
WLmodelActn.optimize()

,fluxes,reduced_costs
EX_h2_e,-10.000000,-0.333333
EX_h2o_e,5.000000,0.000000
EX_co2_e,-3.333333,-0.000000
EX_co_e,0.000000,-0.333333
EX_ac_e,0.000000,0.000000
...,...,...
ctfAB_ac,0.000000,0.000000
ADC,0.000000,0.000000
ACt,0.000000,0.000000
ACTt,0.000000,0.000000


# Test model and visualize in Escher

In [24]:
WLmodelActn.objective = "EX_ac_e"
WLmodelActn.optimize()
#Builder(model = WLmodelActn, map_json = join(data_dir,'Models/AcetonePathway2.json'), reaction_data = solution.fluxes.to_dict())

,fluxes,reduced_costs
EX_h2_e,-10.000000,-0.0
EX_h2o_e,5.000000,0.0
EX_co2_e,-3.333333,-0.0
EX_co_e,0.000000,-0.0
EX_ac_e,0.000000,0.0
...,...,...
ctfAB_ac,0.000000,0.0
ADC,0.000000,0.0
ACt,0.000000,0.0
ACTt,0.000000,0.0


In [ ]:
#Builder(model = WLmodelActn, map_json = join(data_dir,'Models/WL_core_clean.json'), reaction_data = solution.fluxes.to_dict())#
#Builder(model = , map_json = join(data_dir,'Models/AcetonePathway.json'), reaction_data = solution.fluxes.to_dict())

In [ ]:
Builder(model = WLmodelActn, map_json = join(data_dir,'Models/EscherMaps/AcetonePathway.json'), reaction_data = solution.fluxes.to_dict())

In [ ]:
#Builder(model = WLmodelActn)

In [ ]:
Builder(model = WLmodelActn, map_json = join(data_dir,'Models/EscherMaps/WL_core_Actn.json'), reaction_data = solution.fluxes.to_dict())

In [ ]:
WLmodelActn.reactions

In [ ]:
WLmodelActn.reactions.EX_etoh_e

In [ ]:
WLmodelActn.reactions.EX_ac_e

In [ ]:
WLmodelActn.reactions.ATPM.knock_out()

In [ ]:
for x in WLmodelActn.reactions:
    if x.bounds == (0, 0):{
        print(x.id)
    }

In [ ]:
WLmodelActn.reactions.BTOHt.bounds = -1000.0, 1000.0
WLmodelActn.reactions.HYDFDN.bounds = -1000.0, 1000.0
WLmodelActn.reactions.HYDFDi.bounds = -1000.0, 1000.0
WLmodelActn.reactions.ACACT1r.bounds = -1000.0, 1000.0
WLmodelActn.reactions.HACD1.bounds = -1000.0, 1000.0
WLmodelActn.reactions.ECOAH1.bounds = -1000.0, 1000.0
WLmodelActn.reactions.ACOAD1z.bounds = -1000.0, 1000.0
WLmodelActn.reactions.BTCOARx.bounds = -1000.0, 1000.0
WLmodelActn.reactions.ALCD4.bounds = -1000.0, 1000.0
WLmodelActn.reactions.PBUTT.bounds = -1000.0, 1000.0
WLmodelActn.reactions.BUTKr.bounds = -1000.0, 1000.0

solution = WLmodelActn.optimize()
WLmodelActn.optimize()

In [ ]:
WLmodelActn.objective = "EX_etoh_e"
WLmodelActn.optimize()
#WLmodelActn.summary()
#Builder(model = WLmodelActn, map_json = join(data_dir,'Models/EscherMaps/WL_core_Actn.json'), reaction_data = solution.fluxes.to_dict())

# Alternative: build from Moorella model

Builder(model = Moorella_Model)#, map_json = join(data_dir,'Models/p-thermo_acetone_map.json'))

Moorella_Model.metabolites.get_by_id("10fthf_c")

Moorella_Model.metabolites.methf_c

Moorella_Model.metabolites.mlthf_c

Moorella_Model.reactions.TMDS

CO2t
CODH_ACS
PFOR
Moorella_Model.reactions.CODH

#Transport & Exchange:
Moorella_Model.reactions.EX_co_LPAREN_e_RPAREN
Moorella_Model.reactions.EX_co2_LPAREN_e_RPAREN
Moorella_Model.reactions.EX_h2_LPAREN_e_RPAREN
Moorella_Model.reactions.EX_ac_LPAREN_e_RPAREN
Moorella_Model.reactions.COt
Moorella_Model.reactions.CO2t
Moorella_Model.reactions.H2td
Moorella_Model.reactions.ACt2r

#Energy:
Moorella_Model.reactions.HYDFDN
Moorella_Model.reactions.CODH

#Eastern branch:
Moorella_Model.reactions.FDH

#Western branch:
Moorella_Model.reactions.CODH_ACS


WLmodel = Model('Wood_Ljungdahl_Core')

WLmodel.add_reactions([Moorella_Model.reactions.CODH])

WLmodel = Model('Wood_Ljungdahl_Core')

reaction = Reaction('R_3OAS140')
reaction.name = '3 oxoacyl acyl carrier protein synthase n C140 '
reaction.subsystem = 'Cell Envelope Biosynthesis'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default